In [ ]:
# full_spoof_detection_with_manual_stop.py

# === SETUP & IMPORTS ===
import carla
import time
import threading
import pandas as pd
import random
import os
import sys
import numpy as np
import cv2
import torch
import torch.nn as nn
from collections import deque
from time import time as now
import matplotlib.pyplot as plt  # ensure matplotlib is installed

# make sure Carla's PythonAPI is on the path
sys.path.append('D:/carla/PythonAPI')
sys.path.append('D:/carla/PythonAPI/carla')

# ==== CONFIG ====
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEQ_LEN     = 20            # must match your training
THRESHOLD   = 0.5           # sigmoid threshold
VIDEO_DIR   = 'video_new'
FPS         = 20

# ==== MODEL CLASSES ====
class MLPDetector(nn.Module):
    def __init__(self, seq_len, feature_dim):
        super().__init__()
        input_dim = seq_len * feature_dim
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: (batch, seq_len, features)
        x_flat = x.view(x.size(0), -1)
        return self.net(x_flat).squeeze(1)

class LSTMDetector(nn.Module):
    def __init__(self, feature_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(feature_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=0.3)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: (batch, seq_len, features)
        _, (h_n, _) = self.lstm(x)
        h_last = h_n[-1]             # last layer hidden state
        return self.classifier(h_last).squeeze(1)

# ==== LOAD YOUR TRAINED MODEL ====
# Pick the one you trained (MLP or LSTM):
model = LSTMDetector(feature_dim=10, hidden_dim=64, num_layers=2).to(DEVICE)
# model = MLPDetector(seq_len=SEQ_LEN, feature_dim=10).to(DEVICE)

model.load_state_dict(torch.load('nn_spoof_detection_model.pth', map_location=DEVICE))
model.eval()

# ==== CARLA SETUP ====
client      = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world       = client.get_world()
blueprints  = world.get_blueprint_library()
spawn_pts   = world.get_map().get_spawn_points()

# cleanup leftover actors
for a in world.get_actors().filter('*vehicle*'): a.destroy()
for s in world.get_actors().filter('*sensor*'): s.destroy()

# make output dirs
os.makedirs(VIDEO_DIR, exist_ok=True)
os.makedirs(f'{VIDEO_DIR}/frames', exist_ok=True)

# === USER INPUT FOR SPAWN ===
start_i = int(input("Enter START spawn index: "))
spoof_i = int(input("Enter SPOOFED GPS spawn index: "))
start_tf  = spawn_pts[start_i]
spoof_loc = spawn_pts[spoof_i].location

# === SPAWN VEHICLE & AUTOPILOT ===
veh_bp  = blueprints.find('vehicle.lincoln.mkz_2020')
vehicle = world.spawn_actor(veh_bp, start_tf)
vehicle.set_autopilot(True)

# spectator follow thread
running = True
def follow_vehicle():
    spectator = world.get_spectator()
    while running and vehicle.is_alive:
        t = vehicle.get_transform()
        spectator.set_transform(
            carla.Transform(t.transform(carla.Location(x=-6, z=3)), t.rotation)
        )
        time.sleep(0.05)
threading.Thread(target=follow_vehicle, daemon=True).start()

# === GLOBAL STATE & DATA ===
spoofing_active   = False
spoofed_lat, spoofed_lon = spoof_loc.x, spoof_loc.y
frame_id          = 0
video_writers     = {}
data_records      = []
sensor_data       = {'gnss': None, 'imu': None}
sensor_list       = []
feat_buffer       = deque(maxlen=SEQ_LEN)

# === MANUAL STOP THREAD ===
def stop_loop():
    global running
    while running:
        cmd = input("Type 'q' + ENTER to quit → ").strip().lower()
        if cmd == 'q':
            running = False
            print("🛑 Stopping simulation…")
            break
threading.Thread(target=stop_loop, daemon=True).start()

# === RANDOM SPOOF THREAD ===
def random_spoof_loop():
    global spoofing_active, running
    while running:
        time.sleep(random.uniform(5,15))
        spoofing_active = True
        print("🚨 Random Spoof ON for 5s")
        time.sleep(5)
        spoofing_active = False
        print("✅ Random Spoof OFF")
threading.Thread(target=random_spoof_loop, daemon=True).start()

# === SENSOR CALLBACKS ===
def gps_callback(data):
    sensor_data['gnss'] = {
        'latitude':  spoofed_lat   if spoofing_active else data.latitude,
        'longitude': spoofed_lon  if spoofing_active else data.longitude
    }

def imu_callback(data):
    sensor_data['imu'] = {
        'accel_x': data.accelerometer.x,
        'accel_y': data.accelerometer.y,
        'accel_z': data.accelerometer.z,
        'gyro_x':  data.gyroscope.x,
        'gyro_y':  data.gyroscope.y,
        'gyro_z':  data.gyroscope.z
    }

# === MAIN RGB CALLBACK WITH SEQUENCE INFERENCE ===
def rgb_callback(image):
    global frame_id
    if not (sensor_data['gnss'] and sensor_data['imu']):
        return

    # convert BGRA→BGR
    arr = np.frombuffer(image.raw_data, dtype=np.uint8)
    arr = arr.reshape((image.height, image.width, 4))
    rgb_img = arr[:, :, :3][:, :, ::-1].copy()

    # compute speed (km/h)
    v = vehicle.get_velocity()
    speed = 3.6 * (v.x**2 + v.y**2 + v.z**2)**0.5

    # assemble feature vector
    ctrl = vehicle.get_control()
    feat = np.array([
        sensor_data['imu']['accel_x'],
        sensor_data['imu']['accel_y'],
        sensor_data['imu']['accel_z'],
        sensor_data['imu']['gyro_x'],
        sensor_data['imu']['gyro_y'],
        sensor_data['imu']['gyro_z'],
        speed,
        ctrl.steer,
        ctrl.throttle,
        ctrl.brake
    ], dtype=np.float32)

    # ground truth flag
    if spoofing_active:
        label, conf, prob = 'spoofed', 1.0, 1.0
    else:
        feat_buffer.append(feat)
        if len(feat_buffer) < SEQ_LEN:
            label, conf, prob = 'normal', 1.0, 0.0
        else:
            seq = torch.tensor(list(feat_buffer),
                               dtype=torch.float32).unsqueeze(0).to(DEVICE)
            with torch.no_grad():
                prob = model(seq).item()
            conf = prob if prob>(1-prob) else (1-prob)
            label = 'spoofed' if prob>THRESHOLD else 'normal'

    print(f"[Frame {frame_id}] {label.upper()} ({conf:.2f})")

    # overlay label
    color = (0,0,255) if label=='normal' else (255,0,0)
    cv2.putText(rgb_img,
                f"{label.upper()} {conf*100:.1f}%",
                (30,50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.0,
                color,
                2)

    # write video & frames
    if 'vid' not in video_writers:
        h,w = image.height, image.width
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        video_writers['vid'] = cv2.VideoWriter(
            f'{VIDEO_DIR}/rgb.avi', fourcc, FPS, (w,h))
    video_writers['vid'].write(rgb_img)
    cv2.imwrite(f'{VIDEO_DIR}/frames/{frame_id:05d}.jpg', rgb_img)

    # log
    rec = {
        'frame_id': frame_id,
        'latitude': sensor_data['gnss']['latitude'],
        'longitude': sensor_data['gnss']['longitude'],
        'label': 'spoofed' if spoofing_active else 'normal',
        'predicted_label': label,
        'spoof_probability': prob
    }
    # add sensor fields
    for i,k in enumerate([
        'accel_x','accel_y','accel_z',
        'gyro_x','gyro_y','gyro_z',
        'speed','steering_angle','throttle','brake'
    ]):
        rec[k] = feat[i]
    data_records.append(rec)

    frame_id += 1

# === ATTACH SENSORS ===
gnss_bp = blueprints.find('sensor.other.gnss')
imu_bp  = blueprints.find('sensor.other.imu')
cam_bp  = blueprints.find('sensor.camera.rgb')
cam_tf  = carla.Transform(carla.Location(x=1.5, z=2.4))

gnss   = world.spawn_actor(gnss_bp, carla.Transform(), attach_to=vehicle)
imu    = world.spawn_actor(imu_bp,  carla.Transform(), attach_to=vehicle)
camera = world.spawn_actor(cam_bp,  cam_tf,                attach_to=vehicle)

gnss.listen(gps_callback)
imu.listen(imu_callback)
camera.listen(rgb_callback)
sensor_list += [gnss, imu, camera]

# === MAIN LOOP ===
try:
    while running and vehicle.is_alive:
        world.tick()
        time.sleep(0.05)
except KeyboardInterrupt:
    running = False
    print("Interrupted")

# === CLEANUP & SAVE CSV ===
all_keys = set().union(*(r.keys() for r in data_records))
cleaned  = [{k: rec.get(k, np.nan) for k in all_keys}
            for rec in data_records]
df = pd.DataFrame(cleaned, columns=sorted(all_keys))
df.to_csv('combined_data_new_test.csv', index=False)
print("💾 combined_data_new_test.csv saved.")

for s in sensor_list:
    s.stop(); s.destroy()
vehicle.destroy()
for w in video_writers.values():
    w.release()

print("✅ Actors cleaned up.")

# === PLOT RESULTS ===
df['ground_truth'] = (df['label']=='spoofed').astype(int)
df['detected']     = (df['predicted_label']=='spoofed').astype(int)
accuracy = (df['ground_truth']==df['detected']).mean()

plt.figure(figsize=(10,4))
plt.plot(df['frame_id'], df['ground_truth'], label='Ground Truth', alpha=0.6)
plt.plot(df['frame_id'], df['detected'],     label='Detected',    alpha=0.6)
plt.xlabel('Frame ID'); plt.ylabel('State (0=normal,1=spoofed)')
plt.title(f"Accuracy: {accuracy*100:.2f}%")
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('ground_vs_detected_new.png')
print(f"📊 Detection accuracy: {accuracy*100:.2f}% — plot saved to ground_vs_detected_new.png")
